<a href="https://colab.research.google.com/github/mocorderos/Water_Impair_Iowa/blob/main/IowaImpairedWaterLists.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**About this data**

### **Summary of impaired waters data analysis**

The dataset includes the last five **impaired waters lists** published biennially by the **Iowa DNR** to comply with the **Clean Water Act** and approved by the **EPA**.

A **segment** is a continuous water body where water quality remains similar throughout. Each segment has **two or more Designated Uses** based on its function.

### **Key context and limitations**
- The **Iowa DNR monitors only half** of the registered water bodies in the state (**data to be confirmed**).
- The analysis covers **five cycles** because data from **2014 and earlier** is not comparable.

### **Criteria for impairment**
A segment is **impaired** if it fails to meet designated use standards due to:
- **Pollutants** (e.g., excess nutrients, chemicals)
- **Biological decline** (e.g., reduced fish or aquatic life diversity)
- **Recreational risks** (e.g., high **E. coli**, algal toxins)

---

### **To-do list**

#### **1. Healthy waters check**
- There is a separate list of **‘healthy’ waters**.
- Need to confirm whether **impaired + healthy waters** equals the total **monitored segments per year**.
- If confirmed, I must calculate the percentage.

#### **2. CycleListed column verification**
- The **cyclelisted** column indicates the year a segment was first declared impaired.
- Need to confirm if impairment is **consecutive** over time.
- **Example**: If a segment has been impaired **from 2016 to 2024** and was first listed in **2004**, can it be said to have been in poor condition for **20 years**?
- This calculation is currently in the **`years_impaired`** column.

#### **3. Cleaning**
- The impairment column needs cleaning—categories should be consolidated and simplified for **clarity and better readability**





#**Imports**

In [ ]:
import pandas as pd

#**Connect with Drive**

In [ ]:
#Connection to the drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load the datasets
df = pd.read_csv("/content/drive/MyDrive/Water/ImpairedWater/impaired/allfivecycles.csv")
# df2 = pd.read_csv("/content/drive/MyDrive/Water/ImpairedWater/Delistings/combined.csv")

#**Impaired list dataset | Cleaning**

In [ ]:
# Check columns and rows | Impaired
df.shape

(3958, 18)

In [ ]:
# Check columns
df.head(1)

,AssessID,SegID,cycle,name,adbCode,type,size,status,use,support,impCode,impairment,listingRationale,dataSource,tmdlPriority,legacyAdbCode,cycleListed,impairmentStatus
0,2553,1,2016,Shrickers Slough,01-MAQ-1,Wetland,140.0,Final,BWW1,PS,5a,Algal Growth: Chlorophyll a,Adverse impacts on plant/animal communities,Ambient monitoring: Long-Term Resource Monito...,Tier IV,IA 01-MAQ-0005-L_0,2004,Continuing


In [ ]:
# Check data type of each column
df.dtypes

,0
AssessID,int64
SegID,int64
cycle,int64
name,object
adbCode,object
type,object
size,float64
status,object
use,object
support,object


In [ ]:
# Convert to lowercase, remove leading/trailing spaces
new_columns = df.columns.str.strip().str.lower()
df.columns = new_columns
# df.head(1)

In [ ]:
# Display missing values
df.isnull().sum()

,0
assessid,0
segid,0
cycle,0
name,0
adbcode,0
type,0
size,0
status,0
use,0
support,0


In [ ]:
# Fill missing values in 'tmdlpriority', 'datasource', and 'legacyadbcode' with 'Unknown'
df['tmdlpriority'] = df['tmdlpriority'].fillna('Unknown')
df['datasource'] = df['datasource'].fillna('Unknown')
df['legacyadbcode'] = df['legacyadbcode'].fillna('Unknown')
# df.isnull().sum()

In [ ]:
# Convert 'cycle', 'segid', and 'size' to integer
df['cycle'] = df['cycle'].astype(int)
df['segid'] = df['segid'].astype(int)
df['size'] = df['size'].astype(int)
df.dtypes

,0
assessid,int64
segid,int64
cycle,int64
name,object
adbcode,object
type,object
size,int64
status,object
use,object
support,object


In [ ]:
# Add two columns that provide descriptions and classifications for the "use" column.
designated_uses_map = {
    'A1': ('Recreational Uses', 'Swimming and water skiing'),
    'A2': ('Recreational Uses', 'Fishing and shoreline activities'),
    'A3': ('Recreational Uses', 'Wading or playing in the water'),
    'BWW1': ('Aquatic Life Uses', 'Sport fish'),
    'BWW2': ('Aquatic Life Uses', 'Small perennial streams, non-game fish'),
    'BWW3': ('Aquatic Life Uses', 'Intermittent pools, non-game fish'),
    'BLW': ('Aquatic Life Uses', 'Impoundments supporting lake communities'),
    'BCW1': ('Protect Aquatic Life', 'Cold water habitat, diverse species'),
    'BCW2': ('Protect Aquatic Life', 'Small cold-water streams, no trout'),
    'C': ('Drinking Water Uses', 'Potable water source waters'),
    'HH': ('Other Uses', 'Fish harvest for consumption'),
    'OIW': ('Other Uses', 'Outstanding state resource water'),
    'GenUse': ('General Use', 'General use')
}

df['designateduses'] = df['use'].map(lambda x: designated_uses_map.get(x, ('Unknown', 'Unknown'))[0])
df['usedescription'] = df['use'].map(lambda x: designated_uses_map.get(x, ('Unknown', 'Unknown'))[1])
# df.head(1)

In [ ]:
# Add two columns that provide descriptions and classifications for the "tmdlpriority" column.
tmdlpriority_map = {
    'N/A': 'N/A',
    'Tier I': 'High impact, low complexity/cost',
    'Tier II': 'High impact, high complexity/cost',
    'Tier III': 'Low impact, low complexity/cost',
    'Tier IV': 'Low impact, high complexity/cost'
}

df['tmdlprioritydescription'] = df['tmdlpriority'].map(lambda x: tmdlpriority_map.get(x, 'Unknown'))
# df.head(1)


In [ ]:
# Create a new column to determine how many years a segment has been impaired since its 'cyclelisted' year
df['years_impaired'] = 2024 - df['cyclelisted']
df[['cyclelisted', 'years_impaired']].head()


,cyclelisted,years_impaired
0,2004,20
1,2004,20
2,2014,10
3,2012,12
4,2004,20


In [ ]:
# Display columns name
list(df.columns)

['assessid',
 'segid',
 'cycle',
 'name',
 'adbcode',
 'type',
 'size',
 'status',
 'use',
 'support',
 'impcode',
 'impairment',
 'listingrationale',
 'datasource',
 'tmdlpriority',
 'legacyadbcode',
 'cyclelisted',
 'impairmentstatus',
 'designateduses',
 'usedescription',
 'tmdlprioritydescription',
 'years_impaired']

In [ ]:
# New shape
df.shape

(3958, 22)

In [ ]:
# unique segments all type
unique_segid_count = df['segid'].nunique()
unique_segid_count

694

#**Analysis**

####**How many segments are there per type and per year?**


In [ ]:
segments_per_type_year = df.groupby(['cycle', 'type'])['segid'].nunique().reset_index()
segments_per_type_year = segments_per_type_year.sort_values(by=['type', 'cycle'], ascending=[True, True])
segments_per_type_year


,cycle,type,segid
0,2016,Lake,86
4,2018,Lake,92
8,2020,Lake,88
12,2022,Lake,88
16,2024,Lake,89
1,2016,Reservoir,7
5,2018,Reservoir,7
9,2020,Reservoir,3
13,2022,Reservoir,3
17,2024,Reservoir,3


####**How many segments are there per type?**

In [ ]:
#Count segments per type across all years
segments_per_type = df.groupby('type')['segid'].nunique().reset_index()
segments_per_type = segments_per_type.sort_values(by='segid', ascending=False)
segments_per_type

,type,segid
2,River,555
0,Lake,127
1,Reservoir,7
3,Wetland,5


# **Rivers**

In [ ]:
# # Filter only river segments
# df_rivers = df[df['type'] == 'River']

In [ ]:
# # number of columns and rows - river's dataset
# df_rivers.shape

In [ ]:
# # Count unique river segments --- all cycles
# total_unique_river_segments = df_rivers['segid'].nunique()
# print("Total unique river segments:", total_unique_river_segments)

In [ ]:
# # Identify river segments present in all five cycles
# river_segments_consecutive = df_rivers.groupby('segid')['cycle'].nunique().reset_index()
# consecutive_river_segments = river_segments_consecutive[river_segments_consecutive['cycle'] == 5]
# print("Number of river segments consistently present in all five cycles:", len(consecutive_river_segments))

In [ ]:
# Create dataset for river segments consistently present in all five cycles
df_rivers = df[df['type'] == 'River']
river_segments_consecutive = df_rivers.groupby('segid')['cycle'].nunique().reset_index()
consecutive_river_segments = river_segments_consecutive[river_segments_consecutive['cycle'] == 5]

In [ ]:
# Count unique segments
unique_segments_count = deteriorated_river_data['segid'].nunique()
print("Number of unique river segments present in all five cycles:", unique_segments_count)

Number of unique river segments present in all five cycles: 443


In [ ]:
# As percentage of total.
total_river_segments = df_rivers['segid'].nunique()
representation_percentage = (unique_segments_count / total_river_segments) * 100
print(f"Percentage of total river segments that are present in all five cycles: {int(representation_percentage)}%")

Percentage of total river segments that are present in all five cycles: 79%


In [ ]:
# # Most frequent uses
use_counts = deteriorated_river_data.groupby(['use', 'usedescription'])['segid'].nunique().reset_index()
total_use_segments = use_counts['segid'].sum()
use_counts['percentage'] = ((use_counts['segid'] / total_use_segments) * 100).astype(int)
use_counts = use_counts.sort_values(by='segid', ascending=False)
print("Most frequent uses in deteriorated river segments:")
print(use_counts)

Most frequent uses in deteriorated river segments:
      use                          usedescription  segid  percentage
0      A1               Swimming and water skiing    300          51
5    BWW2  Small perennial streams, non-game fish     97          16
4    BWW1                              Sport fish     81          13
7      HH            Fish harvest for consumption     35           6
1      A2        Fishing and shoreline activities     29           5
3    BCW1     Cold water habitat, diverse species     27           4
6  GenUse                             General use      6           1
2      A3          Wading or playing in the water      5           0


In [ ]:
# Most common impairments
impairment_counts = deteriorated_river_data['impairment'].value_counts().reset_index()
impairment_counts.columns = ['impairment', 'count']
impairment_counts['percentage'] = ((impairment_counts['count'] / impairment_counts['count'].sum()) * 100).astype(int)
print("Most common impairments in deteriorated river segments:")
print(impairment_counts)

Most common impairments in deteriorated river segments:
                                           impairment  count  percentage
0               Bacteria: Indicator Bacteria- E. coli   1593          55
1       Biological: low aquatic macroinvertebrate IBI    227           7
2   Biological: low fish & invert IBIs- cause unknown    218           7
3                  Fish Consumption Advisory: Mercury    161           5
4                            Biological: low fish IBI    138           4
5                  Fish Kill: Due To Unknown Toxicity     71           2
6                   Fish Kill: Caused By Animal Waste     69           2
7           Biological: loss of native mussel species     60           2
8                                                  pH     54           1
9            Organic Enrichment: Low Dissolved Oxygen     50           1
10                 Temperature: Thermal Modifications     32           1
11              Fish Kill: Caused By Fertilizer Spill     28        

In [ ]:
# Most common TMDL priority levels
tmdl_counts = deteriorated_river_data.groupby(['tmdlpriority', 'tmdlprioritydescription'])['segid'].nunique().reset_index()
tmdl_counts['percentage'] = ((tmdl_counts['segid'] / tmdl_counts['segid'].sum()) * 100).astype(int)
tmdl_counts = tmdl_counts.sort_values(by='segid', ascending=False)
print("Most common TMDL priorities in deteriorated river segments:")
print(tmdl_counts)

Most common TMDL priorities in deteriorated river segments:
  tmdlpriority            tmdlprioritydescription  segid  percentage
1     Tier III    Low impact, low complexity/cost    298          52
2      Tier IV   Low impact, high complexity/cost    245          43
3      Unknown                            Unknown     21           3
0      Tier II  High impact, high complexity/cost      3           0


# **Lake**

In [ ]:
# Create dataset for lake segments consistently present in all five cycles
df_lake = df[df['type'] == 'Lake']
lake_segments_consecutive = df_lake.groupby('segid')['cycle'].nunique().reset_index()
consecutive_lake_segments = lake_segments_consecutive[lake_segments_consecutive['cycle'] == 5]

In [ ]:
# # shape
# df_lake.shape

In [ ]:
# df_lake.head(1)

In [ ]:
# Count unique segments
unique_segments_count = deteriorated_lake_data['segid'].nunique()
print("Number of unique lake segments present in all five cycles:", unique_segments_count)

Number of unique lake segments present in all five cycles: 54


In [ ]:
# Calculate representation of impaired segments
total_lake_segments = df_lake['segid'].nunique()
representation_percentage = (unique_segments_count / total_lake_segments) * 100
print(f"Percentage of total lake segments that are present in all five cycles: {int(representation_percentage)}%")

Percentage of total lake segments that are present in all five cycles: 42%


In [ ]:
# Most frequent uses
use_counts = deteriorated_lake_data.groupby(['use', 'usedescription'])['segid'].nunique().reset_index()
total_use_segments = use_counts['segid'].sum()
use_counts['percentage'] = ((use_counts['segid'] / total_use_segments) * 100).astype(int)
use_counts = use_counts.sort_values(by='segid', ascending=False)
print("Most frequent uses in deteriorated lake segments:")
print(use_counts)

Most frequent uses in deteriorated lake segments:
   use                            usedescription  segid  percentage
0   A1                 Swimming and water skiing     35          49
1  BLW  Impoundments supporting lake communities     21          29
2   HH              Fish harvest for consumption     15          21


In [ ]:
# Most common impairments
impairment_counts = deteriorated_lake_data['impairment'].value_counts().reset_index()
impairment_counts.columns = ['impairment', 'count']
impairment_counts['percentage'] = ((impairment_counts['count'] / impairment_counts['count'].sum()) * 100).astype(int)
print("Most common impairments in deteriorated river segments:")
print(impairment_counts)

Most common impairments in deteriorated river segments:
                                  impairment  count  percentage
0                Algal Growth: Chlorophyll a    116          26
1      Bacteria: Indicator Bacteria- E. coli     70          16
2         Fish Consumption Advisory: Mercury     67          15
3                                         pH     58          13
4                Turbidity: Suspended Solids     41           9
5                                  Turbidity     37           8
6        Turbidity: Secchi Disk Transparency     24           5
7   Organic Enrichment: Low Dissolved Oxygen     10           2
8            Fish Consumption Advisory: PCBs      5           1
9                Algal Growth: Cyanobacteria      4           0
10                           Metals: Mercury      1           0


In [ ]:
# Most common TMDL priority levels
tmdl_counts = deteriorated_lake_data.groupby(['tmdlpriority', 'tmdlprioritydescription'])['segid'].nunique().reset_index()
tmdl_counts['percentage'] = ((tmdl_counts['segid'] / tmdl_counts['segid'].sum()) * 100).astype(int)
tmdl_counts = tmdl_counts.sort_values(by='segid', ascending=False)
print("Most common TMDL priorities in deteriorated lake segments:")
print(tmdl_counts)

Most common TMDL priorities in deteriorated lake segments:
  tmdlpriority            tmdlprioritydescription  segid  percentage
3      Tier IV   Low impact, high complexity/cost     31          40
1      Tier II  High impact, high complexity/cost     23          29
0       Tier I   High impact, low complexity/cost     20          25
4      Unknown                            Unknown      2           2
2     Tier III    Low impact, low complexity/cost      1           1
